In [3]:
%pip install matplotlib
%pip install ipywidgets
%pip install jupyterlab_widgets
%pip install numpy
%pip install jupyter_contrib_nbextensions

import matplotlib.pyplot as plt
from ipywidgets import widgets,interact
import numpy as np
import inspect
from IPython.display import display,clear_output, HTML


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
def initDelegations(vNumber, dNumber):
    delegations = []
    for i in range(dNumber):
        delegations += [np.zeros(vNumber)]
    return delegations

def getValidatorLabels(vNumber):
    labels = []
    for i in range(vNumber):
        labels += [ f"V{i}" ]
    return labels

def plot(**fields):
    parsed_delegations = []
    
    to_validator=None
    validatorNumber=0
    delegatorNumber=0
    nakamoto_coefficient = 0
    total_reward = 0
    for k1, v1 in locals()['fields'].items():
        if "_to" in k1:
            del_id = k1
            del_id = del_id.replace("d","")
            del_id = del_id.replace("_to","")
            del_id = int(del_id)
            to_validator=[del_id,v1]
            if v1 > validatorNumber:
                validatorNumber = v1
            if del_id > delegatorNumber:
                delegatorNumber = del_id
        if "coefficient" in k1:
            nakamoto_coefficient = v1
        if "reward" in k1:
            total_reward = v1
    labelReward = ['Reward']
    PR = [total_reward *(1-nakamoto_coefficient)]
    NB = [total_reward - PR[0]]
    
    validatorNumber+=1
    delegatorNumber+=1
    delegations = initDelegations(validatorNumber, delegatorNumber)
    labels = getValidatorLabels(validatorNumber)
    for k1, v1 in locals()['fields'].items():
        
        if "_to" in k1:
            del_id = k1
            del_id = del_id.replace("d","")
            del_id = del_id.replace("_to","")
            del_id = int(del_id)
            to_validator=[del_id,v1]
            if v1 > validatorNumber:
                validatorNumber = v1
            if del_id > delegatorNumber:
                delegatorNumber = del_id
        if "_amount" in k1:
            delegations[to_validator[0]][to_validator[1]] = v1

    fig, axs = plt.subplots(1,4,width_ratios=[1,1,1, 2])
    fig.set_size_inches(30.5, 5.5)
    offsets = np.zeros(validatorNumber)
    total_stake = np.sum(delegations)

    for d in range(len(delegations)):   
        delegation_vp = delegations[d]/total_stake*100
        axs[0].bar(labels, delegations[d]/total_stake*100, bottom=offsets, label="D"+str(d))
        validatorID = np.argwhere(delegations[d] != 0)
        axs[0].text(validatorID[0], offsets[validatorID[0]] + delegation_vp[validatorID[0]]/2, "D"+str(d), ha='center')
        offsets += delegations[d]/total_stake*100
    
    # Setting title and labels
    axs[0].set_title('Validator Stake (%)')
    axs[0].set_ylabel('Voting Power (%)')

    
    # Plotting the stacked bar chart
    axs[1].bar(labelReward, PR, label='Proportional Reward')
    axs[1].bar(labelReward, NB, bottom=PR, label='Nakamoto Bonus')

    # Setting title and labels
    axs[1].set_title('Block Reward (Coins)')
    axs[1].legend()
    
    
    PR_j = []
    NB_j = []
    validator_stake = np.zeros(validatorNumber)
    for d in delegations:
        validator_stake += d

    for j in range(len(validator_stake)):
        PR_j.append(validator_stake[j] * PR[0] / sum(validator_stake))
        NB_j.append(NB[0] / len(validator_stake))
        
    # Plotting the stacked bar chart
    axs[2].bar(labels, PR_j, label='Proportional Reward')
    axs[2].bar(labels, NB_j, bottom=PR_j, label='Nakamoto Bonus')
    
    #Add y axis space for text
    x1,x2,y1,y2 = axs[2].axis()
    axs[2].axis((x1,x2,y1,y2*1.20))
    
    for val in range(len(labels)):
        axs[2].text(val, PR_j[val]+NB_j[val]+y2/100, f"{PR_j[val]:.2f}+\n{NB_j[val]:.2f}=\n{PR_j[val]+ NB_j[val]:.2f}",ha='center')


    
    # Setting title and labels
    axs[2].set_title('Validator Rewards (Coins)')
    axs[2].legend()

    PR_d = []
    NB_d = []
    labelsDelegators = []
    for d in range(len(delegations)):
        validatorID = np.argwhere(delegations[d] != 0)
        PR_d.append( PR_j[validatorID[0][0]] / validator_stake[validatorID[0][0]] * np.sum(delegations[d]) )
        NB_d.append( NB_j[validatorID[0][0]] / validator_stake[validatorID[0][0]] * np.sum(delegations[d]) )
        labelsDelegators += [f"D{str(d)}"]




    axs[3].bar(labelsDelegators, PR_d, label='Proportional Reward')
    axs[3].bar(labelsDelegators, NB_d, bottom=PR_d, label='Nakamoto Bonus')
    #Add y axis space for text
    x1,x2,y1,y2 = axs[3].axis()
    axs[3].axis((None,None,None,y2*1.2))
    for d in range(len(delegations)):
        axs[3].text(d, PR_d[d]+NB_d[d]+y2/100, f"{PR_d[d]:.2f}+\n{NB_d[d]:.2f}=\n{PR_d[d]+ NB_d[d]:.2f}",ha='center')
    axs[3].set_title('Delegator Rewards (Coins)')
    axs[3].legend()
    plt.show()         
    
    # Compute Nakamoto Coefficient Update
    validators_per_group = validatorNumber//3
    validators_in_low_group = validators_per_group
    if validatorNumber % 3 != 0:
        validators_in_low_group+=1


    validator_stake[::-1].sort()

    VPHigh = validator_stake[0:validators_per_group]
    VPMid = validator_stake[validators_per_group:2*validators_per_group]
    VPLow = validator_stake[2*validators_per_group:2*validators_per_group + validators_in_low_group]

    avgVPHigh = np.sum(VPHigh/total_stake)/len(VPHigh)
    avgVPMid = np.sum(VPMid/total_stake)/len(VPMid)
    avgVPLow = np.sum(VPLow)/total_stake/validators_in_low_group
    
    display(HTML('<h2>Nakamoto Bonus Update</h2>'))
    display(HTML('<h3>Sort validators in 3 groups:</h3>'))
    print(f"VPHigh:{VPHigh}")
    print(f"VPMid:{VPMid}")
    print(f"VPLow:{VPLow}")
    display(HTML('<h3>Compute Average VP per group:</h3>'))
    print(f"avgVPHigh:{avgVPHigh*100}")
    print(f"avgVPMid:{avgVPMid*100}")
    print(f"avgVPLow:{avgVPLow*100}")
    display(HTML('<h3>Update:</h3>'))
    if avgVPHigh >= 3 * avgVPLow:
        display(HTML('<h4>Nakamoto Bonus will <b>increase</b> in the next update</h4>'))
    else:
        if nakamoto_coefficient == 0:
            display(HTML('<h4>Nakamoto Bonus will <b>remain 0</b> in the next update</h4>'))
        else:
            display(HTML('<h4>Nakamoto Bonus will <b>decrease</b> in the next update</h4>'))
    
    
def generate_gui(valNumber, delNumber):
    style = {'description_width': 'initial'}
    slide = widgets.FloatSlider(
    value=0.0,
    min=0,
    max=1.0,
    step=0.01,
    description='Nakamoto Bonus:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style = style,
    layout=widgets.Layout(width='50%')
    )
    style = {'description_width': 'initial'}
    reward = widgets.BoundedIntText(
    value=100,
    min=0,
    max=1000,
    step=1,
    description='Block Reward:',
    disabled=False,
    style = style
    )
    vBox = [reward,slide]
    ui_fields = { "coefficient" : slide , "block_reward" : reward }
    for d in range(delNumber):
        style = {'description_width': 'initial'}
        delegatesTo = widgets.BoundedIntText(
                    value=100,
                    min=0,
                    max=valNumber-1,
                    step=1,
                    description=f'D{d} delegates to validator:',
                    disabled=False,
                    style = style
                )
        delegatedAmount = widgets.BoundedIntText(
                    value=100,
                    min=0,
                    max=1000,
                    step=1,
                    description=f'amount:',
                    disabled=False
                )
        ui_fields[f"d{d}_to"] = delegatesTo
        ui_fields[f"d{d}_amount"] = delegatedAmount
        vBox += [widgets.HBox([delegatesTo,delegatedAmount])]
    sub_gui = widgets.VBox(vBox)
    out = widgets.interactive_output(plot, ui_fields)
    display(sub_gui,out)

display(HTML('<h1>Nakamoto Bonus Simulator</h1>'))
style = {'description_width': 'initial'}
vNumber = widgets.BoundedIntText(
    value=4,
    min=0,
    max=10,
    step=1,
    description='Validator Number  :',
    disabled=False,
    style = style
)
dNumber = widgets.BoundedIntText(
    value=10,
    min=0,
    max=100,
    step=1,
    description='Delegator Number:',
    disabled=False,
    style = style
)

gui = widgets.interactive(generate_gui, valNumber=vNumber, delNumber=dNumber) #Create our interactive graphic with the dropdown as the argument
display(gui)

interactive(children=(BoundedIntText(value=4, description='Validator Number  :', max=10, style=DescriptionStyl…